In [17]:
import pandas as pd
from kss import split_sentences
import joblib

In [126]:
review = pd.read_csv('review_final_preprocessed.csv', encoding='UTF-8')
review

,id,name,date,content
0,A000000192697,[2023어워즈/3년연속] 아누아 어성초 77 수딩 토너 350ml 어워즈 한정 기...,2023.11.30,피부타입 복합성요 토너 없이는 못살아요겨울 강철 추위도 버티게 해주는 저의 최애템...
1,A000000192697,[2023어워즈/3년연속] 아누아 어성초 77 수딩 토너 350ml 어워즈 한정 기...,2023.11.28,다른 토너로 갈아탔다가 환절기 때 피부가 뒤집어지고 너무 안 돌아와서 다시 구매합니...
2,A000000192697,[2023어워즈/3년연속] 아누아 어성초 77 수딩 토너 350ml 어워즈 한정 기...,2023.11.26,전 이거 하나로 흡토 닦토 다 하고 있어요 자극없이 순하고 용량도 많아서 밤마다 화...
3,A000000192697,[2023어워즈/3년연속] 아누아 어성초 77 수딩 토너 350ml 어워즈 한정 기...,2023.11.19,자극없이 순하고 결 정돈하기 좋은 토너예요민감성 지성 복합성 다 두루 쓰기 좋아요
4,A000000192697,[2023어워즈/3년연속] 아누아 어성초 77 수딩 토너 350ml 어워즈 한정 기...,2023.11.11,이제는 제 화장대에 없는게 말이 안되는 재구매템정말이지 공병만 몇 십병 만들었을거에...
...,...,...,...,...
32396,A000000139057,아벤느 이드랑스 딥 모이스트로션 200ml,2021.01.24,후기보고 살짝 콧물 스킨토너 좀 점성이 있는줄 알았는데 막상 받아보니 아주 살짝있어...
32397,A000000139057,아벤느 이드랑스 딥 모이스트로션 200ml,2021.01.22,토너를 오래 찾던 중에 겨울에 쓸만한게 이 제품이 딱이더라구요 저는 겨울되면 얼굴이...
32398,A000000139057,아벤느 이드랑스 딥 모이스트로션 200ml,2021.01.22,속당김이 엄청 심한편이라 수분크림 에센스를 발라도 겨울엔 겉에만 유분기가있지 수분...
32399,A000000139057,아벤느 이드랑스 딥 모이스트로션 200ml,2021.01.11,수부지 피부 지성피부에 아벤느 제품이 좋다고해서 처음 구매해봤는데 타 제품들과 달리...


In [128]:
soothing = set(['진정', '자극', '트러블', '여드름', '순하다', '효과', '좁쌀', '예민', '제거', '나다', '민감',
                     '민감성', '피지', '올라오다', '어성초', '붉다', '티트리', '가라', '앉다' ])

moisturizing = set(['보습', '건조', '건성', '수분', '흡수', '촉촉하다', '타입', '겨울', '지성', '여름',
                         '수분감', '속건조', '복합성' '채우다' '촉촉' '수부지', ])
skin = set(['각질', '모공', '피부결', '크다', '부드럽다', '정돈', '화장솜', '장벽', '탄력', '보호', '보들보들'])

In [147]:
model = joblib.load('./model/model.pk1')
model_pca_5000 = joblib.load('./model/model_pca_5000.pk1')
model_pca_2500 = joblib.load('./model/model_pca_2500.pk1')

tf_idf_vec = joblib.load('./model/tf_idf_vec.pk1')
pca_5000 = joblib.load('./model/pca_5000.pk1')
pca_2500 = joblib.load('./model/pca_2500.pk1')

C:\Users\Joy\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\Joy\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\Joy\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.0.2. Th

In [131]:
def predict_sentiment(sentence):
    vector = tf_idf_vec.transform([sentence]).toarray()
    prediction = model.predict(vector)[0]
    return prediction

sentence = "보습은 별로"
predict_sentiment(sentence)

-1

In [129]:
type_df = pd.DataFrame({"soothing":[0 for i in range(len(review))],
               "moisturizing":[0 for i in range(len(review))],
               "skin":[0 for i in range(len(review))]})

In [134]:
for i in range(len(review)):
    # 디버깅
    if i % 1000 == 0:
        print(i)
    # 돌아오기
    context = split_sentences(review["content"][i])
    for text in context:
        for efficient_type in type_df.columns:
            if any(keyword in text for keyword in eval(efficient_type)):
                type_df.loc[i, efficient_type] = type_df.loc[i, efficient_type] + predict_sentiment(text)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000


In [136]:
type_df.to_csv('review_type_polarity.csv', index=False)

### PCA - PC5000 까지

In [148]:
def predict_sentiment_pca_5000(sentence):
    vector = tf_idf_vec.transform([sentence]).toarray()
    vector = pca_5000.transform(pd.DataFrame(vector))
    prediction = model_pca_5000.predict(vector)[0]
    return prediction

sentence = "보습은 그냥 그래요"
predict_sentiment_pca_5000(sentence)

-1

In [149]:
type_df_pca_5000 = pd.DataFrame({"soothing":[0 for i in range(len(review))],
               "moisturizing":[0 for i in range(len(review))],
               "skin":[0 for i in range(len(review))]})

In [ ]:
for i in range(len(review)):
    # 디버깅
    if i % 1000 == 0:
        print(i)
    # 돌아오기
    context = split_sentences(review["content"][i])
    for text in context:
        for efficient_type in type_df_pca_5000.columns:
            if any(keyword in text for keyword in eval(efficient_type)):
                type_df_pca_5000.loc[i, efficient_type] = type_df_pca_5000.loc[i, efficient_type] + predict_sentiment_pca_5000(text)


### PCA - PC2500 까지

In [151]:
def predict_sentiment_pca_2500(sentence):
    vector = tf_idf_vec.transform([sentence]).toarray()
    vector = pca_2500.transform(pd.DataFrame(vector))
    prediction = model_pca_2500.predict(vector)[0]
    return prediction

sentence = "보습은 그냥 그래요"
predict_sentiment_pca_2500(sentence)

-1

In [152]:
type_df_pca_2500 = pd.DataFrame({"soothing":[0 for i in range(len(review))],
               "moisturizing":[0 for i in range(len(review))],
               "skin":[0 for i in range(len(review))]})

In [153]:
for i in range(len(review)):
    # 디버깅
    if i % 1000 == 0:
        print(i)
    # 돌아오기
    context = split_sentences(review["content"][i])
    for text in context:
        for efficient_type in type_df_pca_2500.columns:
            if any(keyword in text for keyword in eval(efficient_type)):
                type_df_pca_2500.loc[i, efficient_type] = type_df_pca_2500.loc[i, efficient_type] + predict_sentiment_pca_2500(text)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000


In [155]:
type_df_pca_2500.to_csv('review_type_polarity_pca.csv', index=False)

In [156]:
type_df_pca_2500

,soothing,moisturizing,skin
0,0,1,0
1,1,0,0
2,1,0,2
3,2,1,1
4,1,2,0
...,...,...,...
32396,1,3,0
32397,0,3,0
32398,0,1,0
32399,1,1,0
